In [1]:
import sys
sys.path.append('../')
import torch
from fit.datamodules.super_res import MNIST_SResFITDM, CelebA_SResFITDM,BioSRMicrotubules,BioSRFActin
from fit.utils.tomo_utils import get_polar_rfft_coords_2D

from fit.modules.SResTransformerModule import SResTransformerModule

from matplotlib import pyplot as plt
from matplotlib import gridspec
from fit.utils.PSNR import RangeInvariantPsnr as PSNR
# from fit.utils.utils import PSNR
import torch
from torch.nn.functional import interpolate
import numpy as np
from pytorch_lightning import seed_everything
import math
import seaborn as sns
import matplotlib.pyplot as plt

from torch.nn.functional import avg_pool3d
trained_model_path = '/home/aman.kukde/Projects/FourierImageTransformer/models/MNIST/mamba/sum/Mamba_MNIST_wp_1000_sum_L_8_H_16_s_10_subset_False_23-05_12-04-55/epoch=88-step=76540.ckpt'

dataset = trained_model_path.split('/')[-5]
model_type = trained_model_path.split('/')[-4]
loss = trained_model_path.split('/')[-3]
# dataset = "MNIST"
# model_type = 'mamba'
# loss = 'sum'

seed_everything(22122020)

if dataset == "MNIST":
    dm = MNIST_SResFITDM(root_dir="./datamodules/data/",
                            batch_size=32, subset_flag=False)
if dataset == "CelebA":
    dm = CelebA_SResFITDM(root_dir="./datamodules/data/",
                            batch_size=8, subset_flag=False)
if dataset == "Bio":
    dm = BioSRFActin('./', 8)
# dm = BioSRMicrotubules('./', 8)
dm.prepare_data()
dm.setup()
r, phi, flatten_order, order, fc_per_ring = get_polar_rfft_coords_2D(img_shape=dm.gt_shape)
n_heads = 16
d_query = 64
model = SResTransformerModule(img_shape=dm.gt_shape,
                              coords=(r, phi),
                              dst_flatten_order=flatten_order,
                              dst_order=order,
                              loss=loss,
                              w_phi = 1000,
                              lr=0.0001, weight_decay=0.01, n_layers=8,
                              n_heads=n_heads, d_query=d_query,num_shells = 6,
                              model_type = model_type,
                              fc_per_ring=fc_per_ring).cuda()


weights = torch.load(trained_model_path)['state_dict']
model.load_state_dict(weights, strict=True)
model.cuda()
model.eval()

print('Model Loaded')

Seed set to 22122020


Using Full MNIST dataset
Model Loaded


In [22]:
def predict_one_test_batch(no_of_images = 5):
    i = 0
    for fc, (mag_min, mag_max) in dm.test_dataloader():
        i+=1
        if i ==12:
            break

    fc = fc.to('cuda')
    mag_min = mag_min.to('cuda')
    mag_max = mag_max.to('cuda')
    x_input = fc[:, model.dst_flatten_order]


    with torch.no_grad():
        pred = model.sres.forward_inference(x_input,max_seq_length=141)
        # pred = model.sres.forward(x_input)
        print(model.criterion(pred, fc[:,model.dst_flatten_order], mag_min, mag_max))
        
    # fig = plt.figure()
    # lowres = x_input.clone()
    # lowres[:, model.input_seq_length:] = 0
    # pred_img = model.convert2img(fc=pred, mag_min=mag_min, mag_max=mag_max)
    # lowres_img = model.convert2img(fc=lowres, mag_min=mag_min, mag_max=mag_max)
    # gt_img = model.convert2img(fc=fc[:, model.dst_flatten_order], mag_min=mag_min, mag_max=mag_max)



    # lowres_psnr = PSNR(gt_img,lowres_img)
    # pred_psnr = PSNR(gt_img,pred_img)
    # gt_psnr = PSNR(gt_img,gt_img)

    # sns.histplot(pred_psnr.cpu().detach() - lowres_psnr.cpu().detach(), kde=True, color='green', legend= True )
    # fig.legend()
    # plt.savefig('psnr_diff.png')
    # plt.close()
    # for i in [pred_img, lowres_img, gt_img]:
    #     i.cpu().detach().numpy()
    # for sample in range(no_of_images):
    #     fig2 = plt.figure(figsize=(31/2., 10/2.))
    #     gs = gridspec.GridSpec(1,5, width_ratios=[10,0.5, 10, 0.5, 10])
    #     ax0 = plt.subplot(gs[0])
    #     ax1 = plt.subplot(gs[2])
    #     ax2 = plt.subplot(gs[4])
    #     plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0,
    #                         hspace = 0, wspace = 0)

    #     ax0.xaxis.set_major_locator(plt.NullLocator())
    #     ax0.yaxis.set_major_locator(plt.NullLocator())
    #     ax0.imshow(lowres_img[sample].cpu(), cmap='gray')
    #     ax0.set_title('Low-Resolution Input');
    #     ax0.axis('equal');

    #     ax1.xaxis.set_major_locator(plt.NullLocator())
    #     ax1.yaxis.set_major_locator(plt.NullLocator())
    #     ax1.imshow(pred_img[sample].cpu(), cmap='gray')
    #     ax1.set_title('Prediction');
    #     ax1.axis('equal');


    #     ax2.xaxis.set_major_locator(plt.NullLocator())
    #     ax2.yaxis.set_major_locator(plt.NullLocator())
    #     ax2.imshow(gt_img[sample].cpu(), cmap='gray')
    #     ax2.set_title('Ground Truth');
    #     ax2.axis('equal');


    # for i in range(no_of_images):
    #     sample = i
    #     fig = plt.figure(figsize=(31/2., 10/2.))
    #     gs = gridspec.GridSpec(1, 5, width_ratios=[10,0.5, 10, 0.5, 10])
    #     ax0 = plt.subplot(gs[0])
    #     ax1 = plt.subplot(gs[2])
    #     ax2 = plt.subplot(gs[4])
    #     plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0,
    #                         hspace = 0, wspace = 0)
    #     ax0.xaxis.set_major_locator(plt.NullLocator())
    #     ax0.yaxis.set_major_locator(plt.NullLocator())
    #     lowres_psnr = PSNR(gt_img, lowres_img)
    #     ax0.imshow(np.roll(abs(torch.fft.rfftn(lowres_img[sample].cpu(),dim = [0,1])),model.dft_shape[0] // 2 ,0))
    #     ax0.set_title('Low-Resolution Input');
    #     ax0.set_xlabel(f'PSNR: {lowres_psnr[sample].cpu().detach()} dB')
    #     ax0.axis('equal');

    #     ax1.xaxis.set_major_locator(plt.NullLocator())
    #     ax1.yaxis.set_major_locator(plt.NullLocator())
    #     ax1.imshow(np.roll(abs(torch.fft.rfftn(pred_img[sample].cpu(),dim = [0,1])),model.dft_shape[0] // 2,0))
    #     pred_psnr = PSNR(gt_img, pred_img)
    #     ax1.set_title('Prediction');
    #     ax1.set_xlabel(f'PSNR: {pred_psnr[sample].cpu().detach()} dB')
    #     ax1.axis('equal');

    #     ax2.xaxis.set_major_locator(plt.NullLocator())
    #     ax2.yaxis.set_major_locator(plt.NullLocator())
    #     ax2.imshow(np.roll(abs(torch.fft.rfftn(gt_img[sample].cpu(),dim = [0,1])),model.dft_shape[0] // 2,0))
    #     ax2.set_title('Ground Truth');
    #     ax2.set_xlabel(f'PSNR: {gt_psnr[sample].cpu().detach()} dB')
    #     ax2.axis('equal');

predict_one_test_batch(5)

torch.Size([141, 10])


RuntimeError: shape '[32, -1, 2, 50]' is invalid for input of size 45120